# Neural Network with Sklearn for Sentiment Analysis ( MLPClassifier )

In [47]:
# import library yang dibutuhkan

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import string
import re
from collections import Counter
from tqdm import tqdm
!pip install terbilang
from terbilang import Terbilang

# Import sklearn modul

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
# from sklearn.externals import joblib

from gensim.models import Word2Vec
import pickle
import time

#import sastrawi
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#tokenize
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import time

In [48]:
# Preprosses Data


list_stopwords = list(set(stopwords.words('indonesian') + stopwords.words('english') + ['ya', 'yg', 'ga', 'yuk', 'dah']))
stopwords = list_stopwords


kamus_alay = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/new_kamusalay.csv', encoding='Latin-1',names=('alay','baku'))

# Melakukan Cleaning tweet tanpa menghapus stopwords
def text_proses(text):

    # Untuk stemmer kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    if type(text) == np.float64:
        return ""
    temp = re.sub(r'\b(USER|RT|URL)\b', '', text)
    temp = temp.lower()
    temp = re.sub(r"@[A-Za-z0-9_]+","", temp) #Menghilangkan Mention
    temp = re.sub("#[A-Za-z0-9_]+","", temp) #Menghilangkan Hashtag
    temp = re.sub(r'https://\S+', '', temp)
    # Melakukan subtitusi kata alay dengan kata baku
    temps = temp.split()
    for i in range(len(temps)):
        temp = temps[i]
        replacement = kamus_alay[kamus_alay['alay'] == temp]['baku']
        if not replacement.empty:
            temps[i] = replacement.iloc[0]
    temp = " ".join(temp.strip() for temp in temps)
    # temp = " ".join(temp.strip() for temp in temps if temp not in stopwords)

    # Mengubah angka menjadi terbaca
    # temps = temp.split()
    # for i in range(len(temps)):
    #     temp = temps[i]
    #     if temps[i].isdigit() == True :
    #         temps[i] = Terbilang().parse(temps[i]).getresult()
    #     else :
    #         temps[i]
    # temp = " ".join(temps)

    # Melakukan stemming
    tokens = word_tokenize(temp)
    temp = ' '.join([word for word in tokens ])
    temp = stemmer.stem(temp)

    temp = re.sub(r'\\x..', '', temp) #Menghilangkan Emoticon
    temp = re.sub(r'&amp;', 'dan', temp)
    temp = re.sub(r'http\S+', '', temp) #Menghilangkan link
    temp = re.sub(r"www.\S+", "", temp) #Menghilangkan link
    temp = re.sub(r"\\[n|t]", "", temp) #Menghilangkan 'enter' dan 'tab'
    temp = re.sub("[^A-Za-z\s']","", temp) #Menghilangkan yang bukan huruf
    temps = temp.split()
    temp = " ".join(temp.strip() for temp in temps)

    return temp



In [7]:
# Upload Clean_data file

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
df.head(1)

,text,label,pro_text_no_angka,no_stopwords
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal putih ba...


In [11]:
df.pro_text_no_angka.duplicated().sum(), df.pro_text_no_angka.isna().sum()  # Data sudah clean


(0, 0)

In [10]:
df = df.drop_duplicates(subset=['pro_text_no_angka']).reset_index(drop=True)

## membuat vector untuk inputan ke Machine Learning

### 1. Dengan menggunakan Bag of Word ( CountVectorizer )

In [13]:
# Proses Feature extraction Bag of Word ( CountVectorizer )



count_vect = CountVectorizer(ngram_range=(1,1))

print('Feature extraction selesai')

Feature extraction selesai


### Best Model for Bag of words

In [21]:
# Cross Validation dengan df.pro_text_update
data_preprocessed = df.pro_text_no_angka.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.79      0.86      0.83       655
     neutral       0.86      0.72      0.79       216
    positive       0.92      0.91      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.86      0.83      0.84      2182
weighted avg       0.88      0.87      0.87      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       671
     neutral       0.83      0.75      0.79       240
    positive       0.91      0.93      0.92      1271

    accuracy                           0.88      2182
   macro avg       0.86      0.84      0.85      2182
weighted avg       0.88      0.88      0.88      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.85      0.84       696
     neutral       0.85      0.66      0.74       213
    positive       0.91      0

In [ ]:
with open('model_BoW.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('bow_tokenizer.pickle', 'wb') as f:
    pickle.dump(count_vect, f)

In [26]:
# real world test
original_text = '''


 Barusan bgt kelar nntn @filmziarah film lokal pertama yg gue tonton dan berhasil bikin nangis @bewe_bw speechless liat aktingnya @alitalit_

                '''
input_ = text_proses(original_text)
text = count_vect.transform([input_])
result = clf.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : barusan banget selesai tonton film lokal gue tonton hasil bikin menang speechless lihat akting
Sentiment : negative


### 2. Dengan menggunakan Tf-IDF ( TfidfVectorizer )

In [27]:
# Proses Feature extraction Tf-IDF ( TfidfVectorizer )
tfidf2 = TfidfVectorizer(ngram_range=(1,1))
# X_tfidf = tfidf.fit_transform(data_preprocessed)
print('Feature Extraction Selesai')

Feature Extraction Selesai


In [30]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text_no_angka.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

start_ = time.time()
for iteration, data in (enumerate(kf.split(np.array(data_preprocessed)), start=1)):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf2.transform(data_test)
    target_test = y[data[1]]

    clf3 = MLPClassifier(random_state=None, verbose=False, early_stopping=True)
    clf3.fit(data_train, target_train)
    preds = clf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('Lama waktu cross validation :', round(end_ - start_), 'detik')

1it [00:34, 34.15s/it]

Training ke- 1
              precision    recall  f1-score   support

    negative       0.81      0.83      0.82       655
     neutral       0.86      0.73      0.79       216
    positive       0.91      0.92      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.86      0.83      0.84      2182
weighted avg       0.87      0.87      0.87      2182



2it [01:04, 31.82s/it]

Training ke- 2
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       671
     neutral       0.83      0.70      0.76       240
    positive       0.90      0.94      0.92      1271

    accuracy                           0.88      2182
   macro avg       0.86      0.83      0.84      2182
weighted avg       0.88      0.88      0.88      2182



3it [01:28, 28.10s/it]

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.84      0.83       696
     neutral       0.83      0.63      0.72       213
    positive       0.91      0.94      0.92      1273

    accuracy                           0.88      2182
   macro avg       0.86      0.80      0.82      2182
weighted avg       0.88      0.88      0.87      2182



4it [01:51, 26.26s/it]

Training ke- 4
              precision    recall  f1-score   support

    negative       0.80      0.85      0.82       707
     neutral       0.88      0.68      0.77       226
    positive       0.90      0.91      0.91      1248

    accuracy                           0.87      2181
   macro avg       0.86      0.81      0.83      2181
weighted avg       0.87      0.87      0.86      2181



5it [02:17, 27.54s/it]

Training ke- 5
              precision    recall  f1-score   support

    negative       0.83      0.85      0.84       672
     neutral       0.87      0.73      0.79       241
    positive       0.91      0.92      0.92      1268

    accuracy                           0.88      2181
   macro avg       0.87      0.83      0.85      2181
weighted avg       0.88      0.88      0.88      2181

rata-rata akurasi :  0.875045377733118
Lama waktu cross validation : 137.7987790107727


In [ ]:
with open('model_tfidf.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('tfidf_tokenizer.pickle', 'wb') as f:
    pickle.dump(tfidf, f)

In [36]:
# real world test
original_text = '''

Aneh dan makin aneh.. Sesama umat sendiri di usir #PILKADA #pilkadadki2017

                '''
input_ = text_proses(original_text)
text = tfidf2.transform([input_])
result = clf3.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : aneh dan makin aneh sama umat sendiri di usir
Sentiment : negative


### 3. Dengan menggunakan gensim model ( word2vec )

In [37]:
# Preprocess the text (tokenization)
nltk.download('punkt')
data_preprocessed = df.pro_text_no_angka.to_list()
corpus = [nltk.word_tokenize(doc) for doc in data_preprocessed]

# Train Word2Vec model
model = Word2Vec(corpus, vector_size=400, window=5, min_count=1, workers=4)

# Generate document embeddings
X_w2v = [model.wv[doc].mean(axis=0) for doc in corpus]

print('Feature Extraction Selesai')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Feature Extraction Selesai


In [38]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

start_ = time.time()
for iteration, data in (enumerate(kf.split(X_w2v), start=1)):

    data_train = np.array(X_w2v)[data[0]]
    target_train = np.array(y)[data[0]]

    data_test = np.array(X_w2v)[data[1]]
    target_test = np.array(y)[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(200,200) ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)

end_ = time.time()
average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)

print('Lama waktu cross validation :', round(end_ - start_), 'detik')

Training ke- 1
              precision    recall  f1-score   support

    negative       0.74      0.79      0.76       655
     neutral       0.75      0.66      0.70       216
    positive       0.90      0.89      0.90      1311

    accuracy                           0.84      2182
   macro avg       0.80      0.78      0.79      2182
weighted avg       0.84      0.84      0.84      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.79      0.77      0.78       671
     neutral       0.74      0.73      0.73       240
    positive       0.89      0.91      0.90      1271

    accuracy                           0.85      2182
   macro avg       0.81      0.80      0.80      2182
weighted avg       0.84      0.85      0.84      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.77      0.75      0.76       696
     neutral       0.76      0.56      0.64       213
    positive       0.88      0

In [40]:
# test dengan real world case

original_text = '''

	bagi2 sembako gratis tim @basuki_btp itu melanggar Pasal 73 Ayat 1 UU No.10 Thn 2016 #pilkadadki2017


                '''
input_ = text_proses(original_text)
text = [nltk.word_tokenize(input_)]
model_w2v = Word2Vec(text, vector_size=400, window=5, min_count=1, workers=4)
data_w2v = [model_w2v.wv[doc].mean(axis=0) for doc in text]
result = clf.predict(data_w2v)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  bagi bagi sembako gratis tim itu langgar pasal  ayat  undang undang no  tahun
Sentiment :  negative


## 4. Kombinasi Bag of word & Tfidf

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df.pro_text_no_angka, df.label, test_size=0.25, random_state=46, stratify=df.label)



In [71]:
Vectorize = CountVectorizer(ngram_range=(1,1))
X_train_vec = Vectorize.fit_transform(X_train)
X_test_vec = Vectorize.transform(X_test)

X_train_vec

<8181x10249 sparse matrix of type '<class 'numpy.int64'>'
	with 199020 stored elements in Compressed Sparse Row format>

In [67]:
# with open ('Vectorize_kombinasi.pickle','wb') as f:
#   pickle.dump(Vectorize,f)

In [ ]:
# with open ('Vectorize_kombinasi.pickle', 'rb') as f:
#   vectorx = pickle.load(f)

In [72]:
Transformer = TfidfTransformer()

X_train_tfidf = Transformer.fit_transform(X_train_vec)
X_test_tfidf = Transformer.transform(X_test_vec)

X_train_tfidf

<8181x10249 sparse matrix of type '<class 'numpy.float64'>'
	with 199020 stored elements in Compressed Sparse Row format>

In [68]:
# with open ('Transformer_kombinasi.pickle','wb') as f:
#   pickle.dump(Transformer,f)

In [73]:

import time

start_ = time.time()
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),early_stopping=True)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred))

Waktu training - predict : 24.755317449569702
              precision    recall  f1-score   support

    negative       0.82      0.83      0.82       850
     neutral       0.84      0.74      0.78       284
    positive       0.91      0.92      0.91      1593

    accuracy                           0.87      2727
   macro avg       0.85      0.83      0.84      2727
weighted avg       0.87      0.87      0.87      2727



In [74]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text_no_angka.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train_vec = Vectorize.fit_transform(data_train)
    data_train_tfidf = Transformer.fit_transform(data_train_vec)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test_vec = Vectorize.transform(data_test)
    data_test_tfidf = Transformer.transform(data_test_vec)
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(128,128,128),verbose=False, early_stopping=True)
    clf.fit(data_train_tfidf, target_train)
    preds = clf.predict(data_test_tfidf)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)
end_ = time.time()
average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('Lama waktu cross validation :', round(end_ - start_), 'detik')


Training ke- 1
              precision    recall  f1-score   support

    negative       0.79      0.86      0.82       655
     neutral       0.82      0.69      0.75       216
    positive       0.92      0.90      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.84      0.82      0.83      2182
weighted avg       0.87      0.87      0.87      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       671
     neutral       0.83      0.70      0.76       240
    positive       0.90      0.94      0.92      1271

    accuracy                           0.88      2182
   macro avg       0.86      0.83      0.84      2182
weighted avg       0.88      0.88      0.88      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.85      0.78      0.81       696
     neutral       0.77      0.70      0.73       213
    positive       0.89      0

In [75]:
# with open('model_mlp_kombinasi.pickle', 'wb') as f:
#     pickle.dump(clf, f)

with open ('Vectorize_kombinasi.pickle','wb') as f:
  pickle.dump(Vectorize,f)

with open ('Transformer_kombinasi.pickle','wb') as f:
  pickle.dump(Transformer,f)


In [76]:
# real world test
original_text = '''

saya cinta kamu

                '''
input_ = text_proses(original_text)
text = Vectorize.transform([input_])
text = Transformer.transform(text)
result = clf.predict(text)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  saya cinta kamu
Sentiment :  positive


In [78]:
# dir(Transformer)
Transformer.get_feature_names_out().shape

(10566,)

In [79]:
Vectorize.get_feature_names_out().shape

(10566,)